In [4]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')
dataset_dir = '/content/drive/MyDrive/Capstone Project ML/donateacry_corpus_cleaned_and_updated_data/'

# Inisialisasi list untuk menyimpan fitur audio dan label
audio_features = []
labels = []

# Menentukan panjang maksimum fitur (jumlah frame MFCC)
max_length = 10000  # Sesuaikan panjang sesuai kebutuhan Anda

# Melakukan iterasi pada setiap folder (kategori)
for category in os.listdir(dataset_dir):
    category_dir = os.path.join(dataset_dir, category)
    for filename in os.listdir(category_dir):
        if filename.endswith(".wav"):
            filepath = os.path.join(category_dir, filename)
            # Membaca file audio
            audio, sr = librosa.load(filepath, sr=None)  # sr=None agar sampling rate asli digunakan
            # Ekstraksi fitur audio (misalnya, MFCC)
            mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
            # Zero-padding untuk menyamakan panjang fitur
            if mfccs.shape[1] < max_length:
                pad_width = max_length - mfccs.shape[1]
                mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
            else:
                mfccs = mfccs[:, :max_length]
            # Menyimpan fitur dan label
            audio_features.append(mfccs)
            labels.append(category)

# Konversi label ke angka
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split data menjadi data pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(audio_features, labels, test_size=0.2, random_state=42)

# Konversi data ke bentuk array NumPy
X_train = np.array(X_train)
X_test = np.array(X_test)

# Membangun model RNN dengan LSTM
model = keras.Sequential([
    layers.LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.BatchNormalization(),
    layers.LSTM(64, return_sequences=True),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('val_accuracy') >= 0.85):
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

callbacks = myCallback()
# Latih model
model.fit(
    X_train,  # Input data for training
    y_train,  # Target data for training
    epochs=50,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks = [callbacks]# Validation data
)

# Evaluasi model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

ModuleNotFoundError: No module named 'librosa'

In [4]:
model.save('/content/drive/MyDrive/Capstone Project ML/babycrymodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
# Make predictions on the test set
predictions = model.predict(X_test)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Convert numeric labels back to original labels using the label encoder
predicted_category_labels = label_encoder.inverse_transform(predicted_labels)

# Display some predictions
for i in range(10):  # Display predictions for the first 10 samples
    print(f"True Label: {label_encoder.classes_[y_test[i]]}, Predicted Label: {predicted_category_labels[i]}")

10/10 [==============================] - 0s 11ms/step
True Label: belly_pain, Predicted Label: belly_pain
True Label: discomfort, Predicted Label: tired
True Label: hungry, Predicted Label: hungry
True Label: discomfort, Predicted Label: discomfort
True Label: discomfort, Predicted Label: discomfort
True Label: tired, Predicted Label: tired
True Label: burping, Predicted Label: burping
True Label: burping, Predicted Label: burping
True Label: discomfort, Predicted Label: discomfort
True Label: discomfort, Predicted Label: discomfort
